In [1]:
rm( list=ls() )  #remove all objects
gc()             #garbage collection

require("data.table")

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,604528,32.3,1292120,69.1,1292120,69.1
Vcells,1086897,8.3,8388608,64.0,1632674,12.5


Loading required package: data.table



In [2]:
setwd("C:/_MCD/Labo1/code")  #Establezco el Working Directory

In [3]:
#cargo el dataset
dataset  <- fread("./datasets/competencia2_2022_limpieza1.csv.gz")   #donde entreno

#ordeno el dataset
setorder( dataset,  foto_mes )


# ctrx_quarter

In [4]:
promMes <- dataset[ , .(prom_ctrx_quarter=mean(ctrx_quarter)), by = .(foto_mes)]

In [5]:
dataset_nuevo <- merge(dataset,promMes)
dataset_nuevo[sample(.N, 3)] 

foto_mes,numero_de_cliente,active_quarter,cliente_vip,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,mactivos_margen,⋯,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria,prom_ctrx_quarter
<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<int>,<int>,<dbl>,<chr>,<dbl>
202001,112986945,1,0,51,38,7063.40,85281.11,276.13,-619.99,⋯,0,-2017.03,0,766,0.00,0,0,222.87,CONTINUA,116.01508
202007,120390875,1,0,51,21,100.81,7911.70,0.00,0.00,⋯,0,NA,NA,631,NA,NA,NA,0.00,CONTINUA,101.99869
202004,115899702,1,0,48,122,2706.95,34700.76,470.56,-355.49,⋯,0,-43455.20,0,1124,41716.38,1,0,2123.13,CONTINUA,99.93116


In [6]:
dataset_nuevo[,ctrx_quarter_estandar:=(ctrx_quarter/prom_ctrx_quarter)]
dataset_nuevo[sample(.N, 3)] 

foto_mes,numero_de_cliente,active_quarter,cliente_vip,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,mactivos_margen,⋯,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria,prom_ctrx_quarter,ctrx_quarter_estandar
<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<int>,<dbl>,<int>,<int>,<dbl>,<chr>,<dbl>,<dbl>
202010,106869182,1,0,45,150,2125.32,53905.36,2841.40,-2870.36,⋯,-28236.15,45.11,4113,27133.69,11,0,1407.60,CONTINUA,107.1745,1.278289
202105,92162702,1,0,34,174,2965.81,19261.83,1330.93,727.01,⋯,-64515.00,0.00,4754,48279.46,41,0,14779.80,,119.2678,2.297351
202102,93669850,1,0,39,115,199.71,16768.82,1084.25,-2221.18,⋯,-193227.75,0.00,3440,109304.96,63,0,6381.12,CONTINUA,119.6463,3.568854


In [7]:
v <- c("ctrx_quarter","ctrx_quarter_norm","prom_ctrx_quarter")
#dataset_nuevo[, ..v]

# mcuentas_saldo

In [8]:
dataset_ipc  <- fread("./datasets/IPC_al_202105.csv")

In [9]:
head(dataset_ipc)

foto_mes,IPC_porc,IPC_dec,IPC_acum_actual
<int>,<dbl>,<dbl>,<dbl>
201901,2.8,0.028,2.397943
201902,3.8,0.038,2.332629
201903,4.8,0.048,2.247235
201904,3.2,0.032,2.144308
201905,3.0,0.030,2.077818
201906,2.6,0.026,2.017299


In [10]:
dataset_nuevo <- merge(dataset_nuevo,dataset_ipc)
dataset_nuevo[,mcuentas_saldo_ipc := mcuentas_saldo*IPC_acum_actual]

dataset_nuevo[,mcuentas_saldo_ipc_log := log(mcuentas_saldo_ipc+6500000)]
dataset_nuevo[,mcuentas_saldo_ipc_plus := ifelse(foto_mes==202105,mcuentas_saldo_ipc,mcuentas_saldo*1.1)]


dataset_nuevo[sample(.N, 3)] 

foto_mes,numero_de_cliente,active_quarter,cliente_vip,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,mactivos_margen,⋯,Visa_mpagominimo,clase_ternaria,prom_ctrx_quarter,ctrx_quarter_estandar,IPC_porc,IPC_dec,IPC_acum_actual,mcuentas_saldo_ipc,mcuentas_saldo_ipc_log,mcuentas_saldo_ipc_plus
<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
202011,36402884,1,0,48,279,2943.30,21068.0,182.63,0.00,⋯,0.00,CONTINUA,108.9607,1.0921371,3.0,0.030,1.251822,232677.078,15.72248,204457.836
202001,133837717,1,0,34,86,1660.97,7775.3,1336.77,-150.46,⋯,363.63,CONTINUA,116.0151,0.6292286,1.9,0.019,1.570971,94180.813,15.70170,65945.781
202011,102315011,1,0,51,158,234.35,-2927.3,1615.27,-1765.27,⋯,5841.54,CONTINUA,108.9607,0.5231245,3.0,0.030,1.251822,9924.994,15.68884,8721.284


In [11]:
min(dataset_nuevo[,list(mcuentas_saldo_ipc)])

[1] -6337313

In [12]:
#Estandarizar el saldo no funciono
#promSdMes <- dataset_nuevo[ , .(prom_saldo_ipc=mean(mcuentas_saldo_ipc),sd_saldo_ipc=sd(mcuentas_saldo_ipc)), by = .(foto_mes)]
#dataset_nuevo <- merge(dataset_nuevo,promSdMes)
#dataset_nuevo[,mcuentas_saldo_ipc_estand := (mcuentas_saldo_ipc-prom_saldo_ipc)/sd_saldo_ipc]

# cprestamos_personales

In [16]:
dataset_nuevo[,cprestamos_personales_log:=log(cprestamos_personales+0.1)]
dataset_nuevo[sample(.N, 3)] 

foto_mes,numero_de_cliente,active_quarter,cliente_vip,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,mactivos_margen,⋯,clase_ternaria,prom_ctrx_quarter,ctrx_quarter_estandar,IPC_porc,IPC_dec,IPC_acum_actual,mcuentas_saldo_ipc,mcuentas_saldo_ipc_log,mcuentas_saldo_ipc_plus,cprestamos_personales_log
<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
202001,113967915,1,0,67,123,7184.61,105977.18,318.19,-21.52,⋯,CONTINUA,116.0151,0.2672066,1.9,0.019,1.570971,46206.48,15.69440,32353.96,0.09531018
202105,61004964,1,0,47,245,2664.85,26345.16,1899.29,481.76,⋯,,119.2678,0.5617609,0.0,0.000,1.000000,-4584.96,15.68661,-4584.96,-2.30258509
202009,63486538,1,0,61,229,1741.17,22493.74,209.19,-135.74,⋯,CONTINUA,106.4139,0.8833436,2.8,0.028,1.333200,126816.05,15.70664,104633.68,-2.30258509


### Guardado Final

In [17]:
fwrite( dataset_nuevo,
        file="./datasets/competencia2_2022_limpieza2.csv.gz",
        sep= "," )